# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [40]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [41]:
# load file from weatherpy challenge
file = pd.read_csv("../output_data/cities.csv")
file.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bredasdorp,-34.5322,20.0403,69.80,83,59,6.91,ZA,1613193623
1,1,punta arenas,-53.1500,-70.9167,53.29,66,90,18.41,CL,1613193623
2,2,vao,-22.6667,167.4833,79.66,74,15,7.65,NC,1613193624
3,3,finschhafen,-6.6024,147.8681,85.42,70,85,8.01,PG,1613193624
4,4,arraial do cabo,-22.9661,-42.0278,75.58,88,100,13.91,BR,1613193625


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [42]:
# Set Gmaps with the G-key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' from DF into lists
locations = file[["Lat", "Lng"]]
humidity = file["Humidity"].astype(float)

In [44]:
# Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

# Add heat layer over map 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 1)
fig.add_layer(heat_layer)
#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [45]:
# configuration of vacation conditions
ideal_weather = file.loc[(file['Max Temp']>=77)&(file['Max Temp']<=80)&(file['Wind Speed']<=10)&(file['Cloudiness']==0), :]
ideal_weather

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
126,126,wajir,1.7471,40.0573,77.00,69,0,4.61,KE,1613193665
133,133,tamandare,-8.7597,-35.1047,77.77,84,0,8.55,BR,1613193667
191,191,deniliquin,-35.5333,144.9667,78.01,37,0,3.00,AU,1613193682
275,275,keti bandar,24.1447,67.4497,78.22,32,0,6.06,PK,1613193718
461,461,ipixuna,-1.7625,-48.8043,77.00,94,0,4.61,BR,1613193779
526,526,tikamgarh,24.7450,78.8333,79.34,21,0,6.55,IN,1613193799
527,527,tabou,4.4230,-7.3528,78.12,90,0,6.87,CI,1613193800
548,548,palitana,21.5167,71.8333,78.80,34,0,6.06,IN,1613193805


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [46]:
# create hotel file
hotel_df = ideal_weather.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

# parameters
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

In [47]:
# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    # Get latitudes and longitudes from hotel_df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location
    params["location"] = f"{lat},{lng}"
    
    # build url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print json
    response = requests.get(base_url, params=params).json()
    print(json.dumps(response, indent=4, sort_keys=True))
    
    results = response['results']
    
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
    except (KeyError, IndexError):
        print("Error. Skipping...")
hotel_df

{
    "html_attributions": [],
    "next_page_token": "ATtYBwJxRJQy6CJ3TcUz4UtIap4KYPEh3eVHnZFN7J_fM43DX0sw-2sZeE1XObQtKdLPX3TEZmpdGa-jS-9bt-Ih5h8wdx06SX6kD1aRfolNbqyCDZFsIt57s61P94V5EHNfxQ2F0Bh35frrq4W8svyuby5W9kFoi77f7YHtaVKEpO4JYnoT0U9HySCaMLuxpuQVdxXOFlZHvFvyf5HKp_3z9jxUUIqpZPOXlI5sdkXkqUlveqyNTnEWXe9tjX1tyIDX70VLsefJE8y3ebn9GfQkbJqIBFrvGQgjP52LeUZbDlfnf6JJmKuOA4jtn4sBommgfb2PXVPUrCSGU47OMbTTwJHg5LP7d-Xkdv_HpMq-VltlY905w63NJ6g_83L6YpHA9rOZBouTVyPhofTpMkdbjN0dPWqt6KU_cd_JMcxowdieudfcx0DgdQJT",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 1.7481883,
                    "lng": 40.0565217
                },
                "viewport": {
                    "northeast": {
                        "lat": 1.749626830291502,
                        "lng": 40.0579038802915
                    },
                    "southwest": {
                        "lat": 1.746928869708498,
   

{
    "html_attributions": [],
    "next_page_token": "ATtYBwJDv1oHyIE-zzpIbUe2606Mruuci1xAfYsFVJL-ag7OwvzAdA9ngzrN1U6ZAw8bYR9XU9PfpEUIyyuTO56jrQMPsbfQcV3Ku97Mk_M14py_WIj9sgGgHGZvxpizNth6b7G8M57xvjx_NZnJ0O36PljuAkm5VIzo46OKnssDsjkQHjIY2klvYOLUU86TG8hPgRtxnPe9ebdEThqbwMnh-NTEkWCzqj59FWkKxBY07yLxt4QHgF1Vkv1V1eN3Ck8gedRoqD4h13sTDsc7S69IWpZy6Q_PLQKBUcGJCQ95hkOLf8A5i7IKJjxgDrfoNFHHt1f9MC_M1zYuxk6q9-ykwy9eMhVnoe9z0bYIUyvGszfnKsxhAU-costcib_efILGSvrHGSHPih9rWv4W3gHFqJutLywaBGfhTp8IV-TU_uSqOxENH9eOewgZ",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -8.7393463,
                    "lng": -35.08880319999999
                },
                "viewport": {
                    "northeast": {
                        "lat": -8.738008369708497,
                        "lng": -35.0875026197085
                    },
                    "southwest": {
                        "lat": -8.74070633

{
    "html_attributions": [],
    "next_page_token": "ATtYBwLy6y13p-P2EvrdMFYhlmpnYp22iaXx3GjJsom1HQAHT2NjF7J_YvO6f_fL_r744zYgaLKuiKMZYyxNTGirKBPKIodfcKW3Bk0KbWLL88sc7L1Ke88V2n1F1w9ZuCGuJrsByDHmqBDPg6wG0sE5WmSA2aoGfCnouI0DZLnb4jSyHNnD8EqqQrOFq7vis3cUMtIDoLKSuy4S0gcwhRoBw87kQkI-jypcnhLoGRgU3pTiB7487ZxntWqkPM2pKmN9lXzb1TOj6-DOWBo13LVT8mJaVs9kpbtoJkOiPKnXCCLM7BNoSozkFmwSOJz0EclRlXaYmRDjkEYMogjJvmcYFQma1R6Muk_AJxZSPEviEVxEVHSnWeFHzc9jSe8aWGfyc5QT4OQB_T3ehAax_Esu_zWDqFwAv2rYYwzfXm20isjYRWQCChAe3bCM",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -35.541529,
                    "lng": 144.960483
                },
                "viewport": {
                    "northeast": {
                        "lat": -35.5402908197085,
                        "lng": 144.9616562302915
                    },
                    "southwest": {
                        "lat": -35.54298878029149,


{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 24.1431251,
                    "lng": 67.451786
                },
                "viewport": {
                    "northeast": {
                        "lat": 24.1444917302915,
                        "lng": 67.45317638029151
                    },
                    "southwest": {
                        "lat": 24.1417937697085,
                        "lng": 67.4504784197085
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Haji laloo memon house",
            "opening_hours": {
                "open_now": false
            },
            "place_id": "ChIJn-WTNQZNTTkRbwP5_b_0oq4",
            "plus_code": {
                "compound_code": "4FV2+7P Keti Bandar, Pakistan",


{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 4.4207899,
                    "lng": -7.355918099999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 4.422258430291502,
                        "lng": -7.354633169708498
                    },
                    "southwest": {
                        "lat": 4.419560469708498,
                        "lng": -7.357331130291502
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "hot\u00eal doufoulougou",
            "photos": [
                {
                    "height": 4128,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/115735265576755976712

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,126,wajir,1.7471,40.0573,77.00,69,0,4.61,KE,1613193665,Al-bustan Hotel And Restaurant.
1,133,tamandare,-8.7597,-35.1047,77.77,84,0,8.55,BR,1613193667,Pousada 100 Passos
2,191,deniliquin,-35.5333,144.9667,78.01,37,0,3.00,AU,1613193682,Riviana Motel
3,275,keti bandar,24.1447,67.4497,78.22,32,0,6.06,PK,1613193718,Haji laloo memon house
4,461,ipixuna,-1.7625,-48.8043,77.00,94,0,4.61,BR,1613193779,Casa da Oscarina
5,526,tikamgarh,24.7450,78.8333,79.34,21,0,6.55,IN,1613193799,White House
6,527,tabou,4.4230,-7.3528,78.12,90,0,6.87,CI,1613193800,hotêl doufoulougou
7,548,palitana,21.5167,71.8333,78.80,34,0,6.06,IN,1613193805,Sanderao Bhavan


In [48]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [51]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))